# Minimal character-level language model with a Vanilla Recurrent Neural Network, in Python/NumPy

*Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD License*

In [1]:
import numpy as np

**data I/O**

In [2]:
data = open('input.txt', 'r').read()
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('data has %d charactors, %d unique.' % (data_size, vocab_size))

data has 12849 charactors, 77 unique.


In [3]:
char_to_ix = {ch: i for i, ch in enumerate(chars)}
ix_to_char = {i: ch for i, ch in enumerate(chars)}

**hyperparameters**

In [4]:
hidden_size = 100   # size of hidden layer of neurons
seq_length = 25     # number of steps to unroll the RNN for
learning_rate = 1e-1

**model parameters**

In [5]:
Wxh = np.random.randn(hidden_size, vocab_size) * 0.01   # input to hidden
Whh = np.random.randn(hidden_size, hidden_size) * 0.01  # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size) * 0.01   # hidden to output
bh = np.zeros((hidden_size, 1))   # hidden bias
by = np.zeros((vocab_size, 1))    # output bias

> Wxh(100, 27)

> Whh(100, 100)

> Why(27, 100

> bh(100, 1)

> by(27, 1)

**loss function**

In [6]:
def lossFun(inputs, targets, hprev):
    """
    inputs, target, are both list of integers.
    hprev is Hx1 array of initial hidden state
    returns the loss, gradients on model parameters, and last hidden state
    """
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    loss = 0
    
    # forward pass
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size, 1))   # encode in 1-of-k representation
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh)   # hidden state
        ys[t] = np.dot(Why, hs[t]) + by   # unnormalized log probabilities for next chars
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t]))   # probabilities for next chars
        loss += -np.log(ps[t][targets[t], 0])   # softmax (cross-entropy loss)
        
    # backward pass: compute gradients going backwards
    for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1
        dWhy += np.dot(dy, hs[t].T)
        dby += dy
        dh = np.dot(Why.T, dy) + dhnext
        dhraw = (1 - hs[t] * hs[t]) * dh
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t].T)
        dhnext = np.dot(Whh.T, dhraw)
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam)   # clip to mitigate exploding gradients
    
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(input) - 1]

In [12]:
def lossFun(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # forward pass
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [7]:
def sample(h, seed_ix, n):
    """
    sample a squence of integers from the model
    h is memory state, seed_ix is seed letter for first step
    """
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = []
    for t in range(n):
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        y = np.dot(Why, h) + by
        p = np.exp(y) / np.sum(np.exp(y))
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
    return ixes

> Wxh(100, 27)

> Whh(100, 100)

> Why(27, 100

> bh(100, 1)

> by(27, 1)

In [16]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by)   # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size) * seq_length   # loss at iteration 0

In [17]:
while True:
    # prepare inputs (we're sweeping from left to right in steps seq_length long)
    if p + seq_length + 1 >= len(data) or n==0:
        hprev = np.zeros((hidden_size, 1))   # reset RNN memory
        p = 0   # go from start of data
    inputs = [char_to_ix[ch] for ch in data[p: p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[p+1: p+seq_length+1]]
    
    # sample from the model now and then
    if n % 100 == 0:
        sample_ix = sample(hprev, inputs[0], 200)
        txt = ''.join(ix_to_char[ix] for ix in sample_ix)
        print('----\n %s \n----' % (txt,))
        
    # forward seq_length characters through the net and fetch gradient
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    if n % 100 == 0: print('iter %d, loss: %f' % (n, smooth_loss))   # print progress
        
    # perform parameter uapdata with Adagrad
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by],
                                  [dWxh, dWhh, dWhy, dbh, dby],
                                  [mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -learning_rate * dparam /np.sqrt(mem + 1e-8)   # adagrad update
        
    p += seq_length # move data pointer
    n += 1   # iteration counter

----
 e0l1nWO;~rNchsC=rU=ah~3“”/?5_a:mZ-(ot-”x t(y,.qAg;:vZ21HbR:d[t-5ns:IV(igL(/r:!aOp=2’m?cYDkL~VhoBr’HpUw:WPTCur−Edh?]Hr:zS,L3!bTkvj)e−+KUjvW“Sx,gg(eH(?H,3gx(lWic3)I5Hikss”5.s_Ej,v]xqvy.Ac4)BG)]gj2srkAP] 
----
iter 0, loss: 108.595140
----
  gin,ggiapaegpmepr onruuga rgniiuomtgnag awy.rodgoyhhgtnnsxurnoadg:iltR mtognr ggeiiuaindrlmmi(a)pmbdgrrhrgndgnrndrgnxugsnngind ida eesNdggigeiengnortafugaabgr gviamatid adsiatr ons a puuppai ifigtudx 
----
iter 100, loss: 108.162231
----
 p ee    TSk nriu mr Mmeeo erS sTje   nSfor m TioMgho M ST r tmeMiaTrehhSSSdr tM Tpso p d aerg T a dts not S.tS -eSM Sh  M e r ar M TMTb e S tutMSMSSoeyae io r doMT M -n yr MrnnrSgT  e u  Mg m T eSe T  
----
iter 200, loss: 105.849950
----
  vulgr  ortsfwnt luavarfripa f cd toco pa cagu o kr iopcg cow tutobcure thulm:l alftenc  t f drlu)gacclScaf  erIeuusg thig uulashucu ursurc arr rrooo a on  il vurul  purevtuul rr rgf gclinealelluNl ,e 
----
iter 300, loss: 103.103404
----
 s td Ictouvexn inmpnl r d baoo/ Mce 

KeyboardInterrupt: 